## read result files
These files have been generated with the [reference coreference scorer script](https://github.com/conll/reference-coreference-scorers).

In [21]:
import pandas as pd
import os 
import re
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.1f}'.format

In [22]:
def extract_totals(fname):
    sout = []
    copy = True
    with open(fname) as fin:
        for line in fin.readlines():
            if line.startswith('====== TOTALS ======='):
                copy = True

            if copy:
                sout.append(line)

            if line.startswith('METRIC'):
                copy = False
    return sout

In [23]:
MENTION_RESULTS_REGEX = re.compile(
    r".*Identification of Mentions: Recall: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*Precision: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*F1: ([0-9.]+)%.*", re.DOTALL)

COREF_RESULTS_REGEX = re.compile(
    r".*(?:Coreference|BLANC): Recall: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*Precision: \([0-9.]+ / [0-9.]+\) ([0-9.]+)%\s*F1: ([0-9.]+)%.*", re.DOTALL)

METRIC_REGEX = re.compile(
    r".*METRIC (.*):.*")



def get_results_dict(lines):
    results = {}

    metric = ''
    for s in lines:
        metric_match = re.match(METRIC_REGEX, s)
        if(metric_match):
            metric, = metric_match.groups()
            results[metric] = {}

        mention_match = re.match(MENTION_RESULTS_REGEX, s)
        if(mention_match):
            r, p, f1 = mention_match.groups()
            results[metric]['mention_rec'] = r
            results[metric]['mention_prec'] = p
            results[metric]['mention_f1'] = f1

        coref_match = re.match(COREF_RESULTS_REGEX, s)
        if(coref_match):
            r, p, f1 = coref_match.groups()
            results[metric]['coref_rec'] = r
            results[metric]['coref_prec'] = p
            results[metric]['coref_f1'] = f1
            
    return results

In [24]:
basepath = '/home/dafne/shared/FilterBubble/coref-resolution/pipeline/results_nosing'
os.listdir(basepath)

['dev_coref.txt',
 'dev_e2e.txt',
 'dev_entity.txt',
 'news_coref.txt',
 'news_e2e.txt',
 'news_entity.txt',
 'news_goldmentions_coref.txt',
 'news_goldmentions_e2e.txt',
 'news_goldmentions_entity.txt',
 'news_goldparsementions_coref.txt',
 'news_goldparsementions_entity.txt']

In [25]:
results = {}
for fname in os.listdir(basepath):
    expname = fname.split('.')[0]#.split('_')[-1]
    expname = expname.split('_')
    dataset = expname[0]
    tool = expname[-1]
    pipeline = expname[1] if len(expname)>2 else 'endtoend'
    sout = extract_totals(os.path.join(basepath, fname))
    results[(dataset, pipeline, tool)] = get_results_dict(sout)

In [26]:
for exp in results:
    if 'ceafe' in results[exp]:
        results[exp]['conll'] = {}
        for m in ['coref_rec', 'coref_prec','coref_f1']:
            results[exp]['conll'][m] = (float(results[exp]['muc'][m]) + 
                                                 float(results[exp]['bcub'][m]) + 
                                                 float(results[exp]['ceafe'][m]))/3

In [27]:
df_results = pd.DataFrame.from_dict({(*outerKey, innerKey): values for outerKey, innerDict in results.items() 
          for innerKey, values in innerDict.items()}, orient='index')
for c in df_results.columns:
    df_results[c] = df_results[c].astype(float)
    

df_results = df_results.sort_index()
df_results

mention_rec  mention_prec  mention_f1  \
dev  endtoend          coref  bcub          31.8          35.3        33.5   
                              blanc         31.8          35.3        33.5   
                              ceafe         31.8          35.3        33.5   
                              ceafm         31.8          35.3        33.5   
                              conll          nan           nan         nan   
                              muc           31.8          35.3        33.5   
                       e2e    bcub          25.1          37.3        30.0   
                              blanc         25.1          37.3        30.0   
                              ceafe         25.1          37.3        30.0   
                              ceafm         25.1          37.3        30.0   
                              conll          nan           nan         nan   
                              muc           25.1          37.3        30.0   
                       entity bcub          50.3          66.1        57.1   
                              blanc         50.3          66.1        57.1   
                              ceafe         50.3          66.1        57.1   
                              ceafm         50.3          66.1        57.1   
                              conll          nan           nan         nan   
                              muc           50.3          66.1        57.1   
news endtoend          coref  bcub          45.9          54.8        49.9   
                              blanc         45.9          54.8        49.9   
                              ceafe         45.9          54.8        49.9   
                              ceafm         45.9          54.8        49.9   
                              conll          nan           nan         nan   
                              muc           45.9          54.8        49.9   
                       e2e    bcub          58.4          89.8        70.8   
                              blanc         58.4          89.8        70.8   
                              ceafe         58.4          89.8        70.8   
                              ceafm         58.4          89.8        70.8   
                              conll          nan           nan         nan   
                              muc           58.4          89.8        70.8   
                       entity bcub          48.5          76.8        59.4   
                              blanc         48.5          76.8        59.4   
                              ceafe         48.5          76.8        59.4   
                              ceafm         48.5          76.8        59.4   
                              conll          nan           nan         nan   
                              muc           48.5          76.8        59.4   
     goldmentions      coref  bcub          57.4          68.1        62.3   
                              blanc         57.4          68.1        62.3   
                              ceafe         57.4          68.1        62.3   
                              ceafm         57.4          68.1        62.3   
                              conll          nan           nan         nan   
                              muc           57.4          68.1        62.3   
                       e2e    bcub          90.1         100.0        94.8   
                              blanc         90.1         100.0        94.8   
                              ceafe         90.1         100.0        94.8   
                              ceafm         90.1         100.0        94.8   
                              conll          nan           nan         nan   
                              muc           90.1         100.0        94.8   
                       entity bcub          48.7          77.4        59.8   
                              blanc         48.7          77.4        59.8   
                              ceafe         48.7          7

In [28]:
df_results_swapped = df_results.swaplevel().sort_index()

In [29]:
# Show mention identication scores, sorted on f1
mention_scores = df_results_swapped.loc[(slice(None), slice(None),'muc',slice(None)), ['mention_rec', 'mention_prec', 'mention_f1']]
mention_scores = mention_scores.sort_values('mention_f1', ascending=False)
mention_scores = mention_scores.sort_index(level=[0,1], sort_remaining=False)
mention_scores

mention_rec  mention_prec  mention_f1
dev  endtoend          muc entity         50.3          66.1        57.1
                           coref          31.8          35.3        33.5
                           e2e            25.1          37.3        30.0
news endtoend          muc e2e            58.4          89.8        70.8
                           entity         48.5          76.8        59.4
                           coref          45.9          54.8        49.9
     goldmentions      muc e2e            90.1         100.0        94.8
                           coref          57.4          68.1        62.3
                           entity         48.7          77.4        59.8
     goldparsementions muc coref          60.3          70.0        64.8
                           entity         53.0          79.4        63.5

In [30]:
sorted_scores = df_results_swapped.sort_values('coref_f1',ascending=False)
sorted_scores = sorted_scores.sort_index(level=[0,1,2], sort_remaining=False)[['coref_rec', 'coref_prec', 'coref_f1']]
sorted_scores

coref_rec  coref_prec  coref_f1
dev  endtoend          bcub  entity       31.3        45.6      37.1
                             e2e          16.3        26.9      20.3
                             coref         9.8        11.8      10.7
                       blanc entity       28.5        32.3      28.7
                             e2e          12.7        19.9      15.4
                             coref         6.3         7.8       7.0
                       ceafe entity       31.9        44.8      37.2
                             e2e          18.0        26.6      21.5
                             coref        16.3        17.0      16.6
                       ceafm entity       37.7        49.5      42.8
                             e2e          21.3        31.6      25.4
                             coref        16.0        17.8      16.9
                       conll entity       35.6        48.8      41.2
                             e2e          18.6        28.5      22.5
                             coref        11.4        12.6      11.9
                       muc   entity       43.7        56.1      49.2
                             e2e          21.4        31.9      25.6
                             coref         8.0         9.1       8.5
news endtoend          bcub  e2e          44.8        78.4      57.0
                             entity       34.3        61.2      44.0
                             coref        25.5        32.1      28.4
                       blanc e2e          36.5        76.8      49.2
                             entity       26.4        51.1      34.0
                             coref        22.5        30.9      26.0
                       ceafe e2e          48.9        71.9      58.2
                             entity       37.0        57.8      45.1
                             coref        29.8        31.8      30.8
                       ceafm e2e          50.8        78.2      61.6
                             entity       41.4        65.5      50.7
                             coref        32.9        39.3      35.8
                       conll e2e          48.5        77.1      59.5
                             entity       37.4        61.4      46.4
                             coref        27.2        32.3      29.5
                       muc   e2e          51.7        81.0      63.1
                             entity       40.9        65.2      50.3
                             coref        26.2        32.9      29.1
     goldmentions      bcub  e2e          57.6        89.2      70.0
                             entity       34.4        61.2      44.0
                             coref        38.0        48.8      42.7
                       blanc e2e          61.2        79.2      69.0
                             coref        33.6        44.1      38.1
                             entity       26.4        51.6      34.1
                       ceafe e2e          69.6        44.2      54.0
                             entity       36.0        57.7      44.4
                             coref        41.8        45.8      43.7
                       ceafm e2e          60.8        67.5      63.9
                             coref        46.3        54.9      50.2
                             entity       40.9        65.0      50.2
                       conll e2e          60.6        74.0      63.9
                             entity       37.2        61.4      46.3
                             coref        40.9        49.1      44.6
                       muc   e2e          54.7        88.6      67.6
                             entity       41.3        65.4      50.6
                             coref        43.0        52.7      47.3
     goldparsementions bcub  entity       38.8        64.4      48.4
                             coref        41.8        51.0      46.0
                       blanc coref        38.1        48.8      42.7
                             entity